In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test,y_test) = mnist.load_data()
print(x_train.shape, x_test.shape)
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

(60000, 28, 28) (10000, 28, 28)


In [6]:
model= tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
print('model',model.summary())
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               200960    
                                                                 
 dense_5 (Dense)             (None, 64)                16448     
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 10)                330       
                                                                 
Total params: 219,818
Trainable params: 219,818
Non-trainable params: 0
_________________________________________________________________
model None


In [7]:
# 체크포인트 선언
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = 'tmp_checkpoint.ckpt',
                                                save_weights_only=True,
                                                save_best_only=True,
                                                monitor = 'val_loss',
                                                verbose = 1)

In [8]:
# 체크포인트 사용
model.fit(x_train, y_train,
          validation_data=(x_test,y_test),
          epochs=10,
          callbacks=[checkpoint]
          )

Epoch 1/10
1867/1875 [============================>.] - ETA: 0s - loss: 0.2311 - accuracy: 0.9314
Epoch 1: val_loss improved from inf to 0.11777, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2306 - accuracy: 0.9315 - val_loss: 0.1178 - val_accuracy: 0.9639
Epoch 2/10
1867/1875 [============================>.] - ETA: 0s - loss: 0.0957 - accuracy: 0.9701
Epoch 2: val_loss improved from 0.11777 to 0.09666, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0957 - accuracy: 0.9701 - val_loss: 0.0967 - val_accuracy: 0.9691
Epoch 3/10
1868/1875 [============================>.] - ETA: 0s - loss: 0.0653 - accuracy: 0.9802
Epoch 3: val_loss improved from 0.09666 to 0.07990, saving model to tmp_checkpoint.ckpt
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0652 - accuracy: 0.9802 - val_loss: 0.0799 - val_accuracy: 0.9766
Epoch 4/10
1869/1875 [========================

In [9]:
# 모델 체크포인트 로드 전
loss, acc = model.evaluate(x_test, y_test)
print(f'체크 포인트 로드 전 : loss : {loss:.4f}, acc : {acc:.4f}')

313/313 [==============================] - 1s 2ms/step - loss: 0.0813 - accuracy: 0.9803
체크 포인트 로드 전 : loss : 0.0813, acc : 0.9803


In [11]:
# 체크 포인트에서 저장된 weighis를 로드 후 검증
model.load_weights('tmp_checkpoint.ckpt')
loss, acc = model.evaluate(x_test,y_test)
print(f'체크포인트 로드 후 : loss : {loss:.4f},acc : {acc:.4f}')

313/313 [==============================] - 1s 3ms/step - loss: 0.0755 - accuracy: 0.9778
체크포인트 로드 후 : loss : 0.0755,acc : 0.9778


In [12]:
# EarlyStopping
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 3)

In [ ]:
model= tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
print('model',model.summary())
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
# 체크포인트 사용
model.fit(x_train, y_train,
          validation_data=(x_test,y_test),
          epochs=30,
          callbacks=[checkpoint,earlystopping]
          )

Epoch 1/30
1875/1875 [==============================] - ETA: 0s - loss: 0.0142 - accuracy: 0.9954
Epoch 1: val_loss did not improve from 0.07549
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0142 - accuracy: 0.9954 - val_loss: 0.0944 - val_accuracy: 0.9800
Epoch 2/30
1866/1875 [============================>.] - ETA: 0s - loss: 0.0163 - accuracy: 0.9948
Epoch 2: val_loss did not improve from 0.07549
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0163 - accuracy: 0.9948 - val_loss: 0.1015 - val_accuracy: 0.9774
Epoch 3/30
1871/1875 [============================>.] - ETA: 0s - loss: 0.0142 - accuracy: 0.9950
Epoch 3: val_loss did not improve from 0.07549
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0141 - accuracy: 0.9950 - val_loss: 0.1046 - val_accuracy: 0.9766
Epoch 4/30
1864/1875 [============================>.] - ETA: 0s - loss: 0.0138 - accuracy: 0.9957
Epoch 4: val_loss did not improve from 0.07549
1875/1875 [=========

In [22]:
# 학습율 스케줄러
def scheduler(epoch, lr):
  tf.print(f'learning_rate : {lr :.5f}')
  if epoch <5:
    return lr
  else : 
    return lr * tf.math.exp(-0.1)
# 콜백
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [18]:
model= tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
print('model',model.summary())
model.compile(optimizer=tf.keras.optimizers.SGD(), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_12 (Dense)            (None, 256)               200960    
                                                                 
 dense_13 (Dense)            (None, 64)                16448     
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 10)                330       
                                                                 
Total params: 219,818
Trainable params: 219,818
Non-trainable params: 0
_________________________________________________________________
model None


In [19]:
model.optimizer.lr.numpy()

0.01

In [23]:
model.fit(x_train, y_train,
          validation_data=(x_test,y_test),
          epochs=10,
          callbacks=[lr_scheduler])

learning_rate : 0.01000
Epoch 1/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.6602 - accuracy: 0.8198 - val_loss: 0.2971 - val_accuracy: 0.9149 - lr: 0.0100
learning_rate : 0.01000
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2637 - accuracy: 0.9234 - val_loss: 0.2317 - val_accuracy: 0.9329 - lr: 0.0100
learning_rate : 0.01000
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2060 - accuracy: 0.9403 - val_loss: 0.1961 - val_accuracy: 0.9400 - lr: 0.0100
learning_rate : 0.01000
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1689 - accuracy: 0.9510 - val_loss: 0.1589 - val_accuracy: 0.9535 - lr: 0.0100
learning_rate : 0.01000
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1424 - accuracy: 0.9588 - val_loss: 0.1383 - val_accuracy: 0.9586 - lr: 0.0100
learning_rate : 0.01000
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - 